In [ ]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from urllib.parse import quote
from openpyxl import Workbook
from pyquery import PyQuery as pq

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
browser = webdriver.Chrome(options=chrome_options)

wait = WebDriverWait(browser, 10)

def index_page(offset):
    print('正在爬取第', offset, '条')
    try:
        url = 'http://maoyan.com/board/4?offset=' + str(offset)
        browser.get(url)
        return get_products()
    except TimeoutException:
        return index_page(offset)


def get_products():
    html = browser.page_source
    doc = pq(html)
    items = doc('.container .board-wrapper dd').items()
    info_list = []
    for item in items:
        info = []
        info.append(item.find('.board-index').text())
        info.append(item.find('.name').text())
        info.append(item.find('.star').text())
        info.append(item.find('.releasetime').text())
        info.append(item.find('.integer').text())
        info.append(item.find('.fraction').text())
        info.append(item.find('.board-img').attr('data-src'))
        info_list.append(info)
        print(info)
    return info_list


def main():
    wb = Workbook()
    ws1 = wb.active
    ws1.title = 'TOP100榜'
    for i in range(10):
       info_list = index_page(i*10)
       for row in info_list:
           ws1.append(row)
    wb.save('TOP100榜.xlsx')
    browser.close()


if __name__ == '__main__':
    main()
